In [1]:
# import the imbalanced-learn library
# install it with ! pip install imbalanced-learn
import imblearn

In [2]:
# import the scikit-learn library
# install it with ! pip install scikit-learn
import sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, cross_validate

In [3]:
# import other necessary libraries
import numpy as np
import pandas as pd
from collections import Counter

In [4]:
# function to append the averages and
# standard deviations of scores,
# row 11 are the averages and
# row 12 are the stds
def append_avgs(dict):
    avgs = []
    stds = []
    for i in dict:
        avg = np.average(dict[i])
        std = np.std(dict[i])
        avgs.append(avg)
        stds.append(std)
    df = pd.DataFrame(dict)
    df.loc[11] = avgs
    df.loc[12] = stds
    return df

In [5]:
# create an imbalaced toy dataset with 10000 samples and 10 features
# class 0: 7500 samples, class 1: 2500 samples 
X, y = make_classification(n_samples=10000, weights=[0.75], n_features = 10,
                           flip_y=0, random_state=42)

# define the Decision Tree Classifier model
model = sklearn.tree.DecisionTreeClassifier()

# define crossvalidation strategy
# this will perform a stratified 10-fold crossvalidation
# parameter n_splits defines the number of folds
cv = StratifiedKFold(n_splits=10)

# define the scoring dictionary to evaluate the crossvalidation
scoring = {'f1': 'f1', 'precision': 'precision', 'accuracy': 'accuracy',
           'recall': 'recall', 'roc_auc': 'roc_auc'}

# Cross-validation without sampling

In [7]:
# crossvalidate on original data set
scores = cross_validate(model, X, y, scoring=scoring, cv=cv, n_jobs=-1)

**Row 11 contains average scores and 12 contains the standard deviation**

In [8]:
# use previously defined function "append_avgs" to append averages and stds
scores = append_avgs(scores)
scores

,fit_time,score_time,test_f1,test_precision,test_accuracy,test_recall,test_roc_auc
0,0.108466,0.007195,0.820408,0.837500,0.912000,0.804000,0.876000
1,0.103974,0.005416,0.827586,0.839506,0.915000,0.816000,0.882000
2,0.102305,0.005039,0.831068,0.807547,0.913000,0.856000,0.894000
3,0.095506,0.005006,0.843327,0.816479,0.919000,0.872000,0.903333
4,0.088152,0.005148,0.823062,0.818182,0.911000,0.828000,0.883333
5,0.094373,0.004866,0.827586,0.839506,0.915000,0.816000,0.882000
6,0.089898,0.005041,0.847390,0.850806,0.924000,0.844000,0.897333
7,0.085832,0.004910,0.837573,0.819923,0.917000,0.856000,0.896667
8,0.074820,0.004948,0.808679,0.797665,0.903000,0.820000,0.875333
9,0.081238,0.004822,0.835759,0.870130,0.921000,0.804000,0.882000


# Cross-validation with sampling

**We will use _SMOTE_ and _Random Undersampling_ as sampling techniques**

In [9]:
# define SMOTE to oversample minority class to contain
# half the amount of samples in the majority class
# by setting sampling_strategy to 0.5
smote = imblearn.over_sampling.SMOTE(sampling_strategy = 0.5)

# define RandomUnderSampler to undersample majority class
# to contain twice the amount of samples in the minority class
# by setting sampling_strategy to 0.5
under = imblearn.under_sampling.RandomUnderSampler(sampling_strategy = 0.5)

## Cross validation with sampling -- DONE WRONG

### Cross-validation with oversampling -- DONE WRONG

In [10]:
# oversample minority class
X_smote, y_smote = smote.fit_resample(X, y)

In [11]:
print("Distribution in original data set", Counter(y))
print("Distribution in data set after SMOTE", Counter(y_smote))

Distribution in original data set Counter({0: 7500, 1: 2500})
Distribution in data set after SMOTE Counter({0: 7500, 1: 3750})


In [12]:
# crossvalidate on oversampled data set using the crossvalidation technique defined in "cv"
wrong_scores_o = cross_validate(model, X_smote, y_smote, scoring=scoring, cv=cv, n_jobs=-1)

In [13]:
# use previously defined function "append_avgs" to append averages and stds
wrong_scores_o = append_avgs(wrong_scores_o)
wrong_scores_o

,fit_time,score_time,test_f1,test_precision,test_accuracy,test_recall,test_roc_auc
0,0.089884,0.004451,0.864430,0.870270,0.910222,0.858667,0.897333
1,0.124148,0.007586,0.873656,0.880759,0.916444,0.866667,0.904000
2,0.132073,0.006117,0.889182,0.879896,0.925333,0.898667,0.918667
3,0.138672,0.007800,0.861789,0.876033,0.909333,0.848000,0.894000
4,0.080333,0.004428,0.880319,0.877984,0.920000,0.882667,0.910667
5,0.081005,0.005078,0.884097,0.893733,0.923556,0.874667,0.911333
6,0.087266,0.005098,0.885333,0.885333,0.923556,0.885333,0.914000
7,0.135069,0.006123,0.908150,0.881910,0.936889,0.936000,0.936667
8,0.128185,0.005452,0.905318,0.881313,0.935111,0.930667,0.934000
9,0.128115,0.005909,0.923684,0.911688,0.948444,0.936000,0.945333


### Cross-validation with undersampling -- DONE WRONG

In [14]:
# undersample majority class
X_under, y_under = under.fit_resample(X, y)

In [15]:
print("Distribution in original data set", Counter(y))
print("Distribution in data set after undersampling", Counter(y_under))

Distribution in original data set Counter({0: 7500, 1: 2500})
Distribution in data set after undersampling Counter({0: 5000, 1: 2500})


In [16]:
# crossvalidate on undersampled data set using the crossvalidation technique defined in "cv"
wrong_scores_u = cross_validate(model, X_under, y_under, scoring=scoring, cv=cv, n_jobs=-1)

In [17]:
# use previously defined function "append_avgs" to append averages and stds
wrong_scores_u = append_avgs(wrong_scores_u)
wrong_scores_u

,fit_time,score_time,test_f1,test_precision,test_accuracy,test_recall,test_roc_auc
0,0.069667,0.006610,0.884848,0.893878,0.924000,0.876000,0.912000
1,0.072461,0.007538,0.869048,0.862205,0.912000,0.876000,0.903000
2,0.071862,0.007442,0.880952,0.874016,0.920000,0.888000,0.912000
3,0.073257,0.007837,0.884462,0.880952,0.922667,0.888000,0.914000
4,0.072965,0.006358,0.867470,0.870968,0.912000,0.864000,0.900000
5,0.074656,0.005146,0.851927,0.864198,0.902667,0.840000,0.887000
6,0.074196,0.004138,0.902584,0.897233,0.934667,0.908000,0.928000
7,0.073489,0.004388,0.863544,0.879668,0.910667,0.848000,0.895000
8,0.059434,0.004732,0.874751,0.869565,0.916000,0.880000,0.907000
9,0.076630,0.004319,0.843373,0.846774,0.896000,0.840000,0.882000


## Cross validation with sampling -- DONE RIGHT

### Cross-validation with oversampling -- DONE RIGHT

In [18]:
# define the SMOTE oversampling pipeline
smote_steps = [('over', smote), ('model', model)]
smote_pipeline = imblearn.pipeline.Pipeline(steps=smote_steps)

This Pipeline first oversamples the training dataset with SMOTE then fits the model.

In [19]:
# show the SMOTE pipeline
print(smote_pipeline)

Pipeline(steps=[('over', SMOTE(sampling_strategy=0.5)),
                ('model', DecisionTreeClassifier())])


In [20]:
# evaluate the SMOTE pipeline using the crossvalidation technique defined in cv
smote_scores = cross_validate(smote_pipeline, X, y, scoring=scoring, cv=cv, n_jobs=-1)

In [21]:
# use previously defined function "append_avgs" to append averages and stds
smote_scores = append_avgs(smote_scores)
smote_scores

,fit_time,score_time,test_f1,test_precision,test_accuracy,test_recall,test_roc_auc
0,0.194384,0.005174,0.847059,0.830769,0.92200,0.864000,0.902667
1,0.187998,0.004536,0.852140,0.829545,0.92400,0.876000,0.908000
2,0.152889,0.004180,0.842912,0.808824,0.91800,0.880000,0.905333
3,0.206151,0.004577,0.849802,0.839844,0.92400,0.860000,0.902667
4,0.195976,0.005112,0.815686,0.800000,0.90600,0.832000,0.881333
5,0.204993,0.005073,0.839357,0.842742,0.92000,0.836000,0.892000
6,0.191301,0.004694,0.828125,0.809160,0.91200,0.848000,0.890667
7,0.189942,0.004753,0.820000,0.820000,0.91000,0.820000,0.880000
8,0.192600,0.004889,0.827184,0.803774,0.91100,0.852000,0.891333
9,0.185301,0.004841,0.862823,0.857708,0.93100,0.868000,0.910000


### Cross-validation with undersampling -- DONE RIGHT

In [22]:
# define RandomUnderSampler undersampling pipeline
under_steps = [('under', under), ('model', model)]
under_pipeline = imblearn.pipeline.Pipeline(steps=under_steps)

This Pipeline first undersamples the training dataset with RandomUnderSampler then fits the model.

In [23]:
# show outher RandomUnderSampler pipeline
print(under_pipeline)

Pipeline(steps=[('under', RandomUnderSampler(sampling_strategy=0.5)),
                ('model', DecisionTreeClassifier())])


In [24]:
# evaluate the RandomUnderSampler pipeline using the crossvalidation technique defined in cv
under_scores = cross_validate(under_pipeline, X, y, scoring=scoring, cv=cv, n_jobs=-1)

In [25]:
# use previously defined function "append_avgs" to append averages and stds
under_scores = append_avgs(under_scores)
under_scores

,fit_time,score_time,test_f1,test_precision,test_accuracy,test_recall,test_roc_auc
0,0.080154,0.005063,0.861538,0.829630,0.92800,0.896000,0.917333
1,0.084976,0.008919,0.844961,0.819549,0.92000,0.872000,0.904000
2,0.080933,0.008015,0.828794,0.806818,0.91200,0.852000,0.892000
3,0.084340,0.007915,0.833006,0.818533,0.91500,0.848000,0.892667
4,0.079821,0.007884,0.816000,0.816000,0.90800,0.816000,0.877333
5,0.071391,0.004895,0.823529,0.807692,0.91000,0.840000,0.886667
6,0.083737,0.007761,0.857692,0.825926,0.92600,0.892000,0.914667
7,0.068670,0.004894,0.833652,0.798535,0.91300,0.872000,0.899333
8,0.065845,0.004958,0.852336,0.800000,0.92100,0.912000,0.918000
9,0.083408,0.005175,0.836292,0.824903,0.91700,0.848000,0.894000


# Compare scores

In [26]:
# create an empty dataframe to compare the scores
df = pd.DataFrame()

# add the average scores of each sampling strategy
# to the empty dataframe
df = df.append([scores.loc[11], wrong_scores_o.loc[11], wrong_scores_u.loc[11],
           smote_scores.loc[11], under_scores.loc[11]]).reset_index(drop=True)

# rename the indices
df = df.rename(index={0:'NO SAMPLING', 1:'WRONG SMOTE', 2:'WRONG UNDERSAMPLING',
                 3: 'RIGHT SMOTE', 4:'RIGHT UNDERSAMPLING'})

df

,fit_time,score_time,test_f1,test_precision,test_accuracy,test_recall,test_roc_auc
NO SAMPLING,0.092456,0.005239,0.830244,0.829725,0.915000,0.831600,0.8872
WRONG SMOTE,0.112475,0.005804,0.887596,0.883892,0.924889,0.891733,0.9166
WRONG UNDERSAMPLING,0.071862,0.005851,0.872296,0.873946,0.915067,0.870800,0.9040
RIGHT SMOTE,0.190154,0.004783,0.838509,0.824237,0.917800,0.853600,0.8964
RIGHT UNDERSAMPLING,0.078327,0.006548,0.838780,0.814759,0.917000,0.864800,0.8996
